In [260]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

# Cargue de Queries y parametros

In [261]:
city = 'VCP'
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [262]:
margenes = pd.read_csv("Margenes_neto.csv")
retention = pd.read_csv("retention_year.csv")

In [263]:
retention_aux =  retention.copy()

for i,ret in enumerate(retention_aux.values):
    retention_aux.iloc[i,2:] = [(1-(j-ret[-1])/100) if j-ret[-1] >= 0 else 1 for j in ret[2:]]

In [264]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [265]:
query = """
SELECT DISTINCT

c.assignee_ref_id AS "Owner",
cm.member_ref_id AS "customer_id",
--cm.status AS "Estado de la asignacion",
--c.type as "Tipo de asociacion",
--c.flow_type AS "Tipo de carga",
z.value AS "Ciudad"
--p.since AS "Semana"

FROM postgres_growth_offline."growth_clustering.clusters"               c
INNER JOIN postgres_growth_offline."growth_clustering.cluster_members"  cm  ON cm.cluster_id = c.id
INNER JOIN postgres_growth_offline."growth_clustering.periods"          p   ON p.id = c.period_id
INNER JOIN postgres_growth_offline."growth_clustering.zones"            z   ON z.id = c.zone_id

where p.since = DATE_TRUNC('week', date(getdate()) - interval '0 week')
and z.value in ('BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP')
and c.type = 'FARMER_OFFLINE'
and c.status = 'ACTIVE'
and c.enabled = 'true'
"""
data_farming = read_connection_data_warehouse.runQuery(query)
data_farming[["customer_id"]] = data_farming[["customer_id"]].astype(int)

In [266]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    --AND extract(year from bo.submit_date::date) = 2021
    --AND extract(month from bo.submit_date::date) = 6--2021
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
    AND bo.customer_id IN {clientes}
GROUP BY bo.order_id, bo.submit_date
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    DATE(bo.submit_date) AS submit_date,
    bo.order_id,
    bs.addl_product_id AS padre_sku_id,
    CASE
    WHEN bcat2.name = 'Abarrotes & Despensa' THEN 'Abarrotes'
    ELSE bcat2.name END AS cat,
    --bcat.name AS subcat,
    --bs2.name,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
    ELSE (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776 END AS gmv_usd,    
    -- CASE 
    --     WHEN s.site_identifier_value = 'CMX' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'GDL' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'PBC' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'MTY' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'SPO' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'BHZ' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'CWB' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'VCP' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    -- ELSE (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/3776 END AS discount_usd,
    (boi.quantity*foi.step_unit) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd", "cant"]] = data_ventas_aux[["gmv_usd", "cant"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [267]:
check_df = data_ventas_aux.groupby(by=['customer_id']).agg({'submit_date':np.max}).reset_index()
df = data_ventas_aux[data_ventas_aux.customer_id.isin(check_df[check_df.submit_date >= customer_date_desde].customer_id.unique())].groupby(by=['city', 'customer_id', 'submit_date','cat']).agg({'gmv_usd':'sum','cant':'sum'}).reset_index()

In [268]:
#df_ENAE = data_ventas_aux[['city','customer_id','submit_date','order_id', 'gmv_usd']].groupby(by=['city','customer_id','submit_date','order_id'])['gmv_usd'].sum().reset_index()
#df_ENAE.to_csv("info_ENAE.csv", index=False)

# Generacion de df con info by cat by client

In [269]:
df = df.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df['order_id']= df.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df['days_bet'] = (df.submit_date -df.groupby('customer_id').submit_date.transform('min')).dt.days

In [270]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df['res'] = df.groupby(by=['customer_id', 'cat'])['days_bet'].diff()
df_grouped = df.groupby(by=['city', 'customer_id', 'cat']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).reset_index()

In [271]:
#Finally we define everything in a df2 for further processing
df2 = df_grouped.copy()

df2.columns = ['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_cat_usd', 'avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [272]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [273]:
def cat_metrics_2(cat):
    df_cat = df2[df2.cat == cat].copy()
    
    df_cat['adjust_std'] = pd.qcut(df_cat.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
    df_cat[["adjust_std"]] = df_cat[["adjust_std"]].astype(float)
    
    df_cat['adjust_cohort'] = [1 if ((x[13]) - (x[5]+x[7]) <= 0) else  
                 1/(((x[13]) - x[5])/x[7]) if x[7] != 0 else
                 1/(((x[13]) - x[5])/np.round(1+(x[5]*0.7)))
                 for x in df_cat.values]
    
    df_cat["margen"] = [margenes[(margenes.category == x[2]) & (margenes.region_code == x[0])]["net_margin"].unique()[0]/100 for x in df_cat.values]
    #mirar aca
    df_cat['adjust_retention'] = [1 if diff_month(x[11],x[14])+1 >= 10 else 
                                  retention_aux[(retention_aux.category == x[2]) & (retention_aux.region_code == x[0])][str(diff_month(x[11],x[14])+1)].unique()[0] for x in df_cat.values]
    
    df_cat['median_diff_days_adjusted'] = 365/df_cat.median_diff_days
    
    df_cat["churned"] = [0 if ((x[13]) - (x[5]+x[7]*2) <= 0) else 1 for x in df_cat.values]
    
    df_cat[['avg_diff_days','stdev_diff_days']] = df_cat[['avg_diff_days','stdev_diff_days']].fillna(0)

    df_cat['churned_date'] = [(x[11] + datetime.timedelta(days=int(x[5]+x[7]*2))).strftime('%Y-%m-%d') for x in df_cat.values] #1 months
    
    df_cat["CLTV"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_margen"] = df_cat.median_diff_days_adjusted * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_churn"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std
    
    df_cat['cash_margin'] = df_cat.aov_cat_usd * df_cat.margen

    return df_cat

In [274]:
#df2['cat'] = df2['cat'].str.replace('Abarrotes & Despensa', 'Abarrotes')

In [275]:
df3 = pd.DataFrame()

for cat in df2.cat.unique():
   print(cat)
   if cat not in ('Mesa & Cocina','Insumos Internos Cat','Despensa'):
      df3 = df3.append(cat_metrics_2(cat))
   print(f'Done {cat}')

Descartáveis


Done Descartáveis
Frutas e Verduras
Done Frutas e Verduras
Laticínios e Ovos
Done Laticínios e Ovos
Limpeza e Higiene
Done Limpeza e Higiene
Mercearia
Done Mercearia
Bebidas
Done Bebidas
Carnes, Aves e Peixes
Done Carnes, Aves e Peixes
Congelados
Done Congelados


In [276]:
df3.head(10)

,city,customer_id,cat,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders,aov_cat_usd,...,adjust_cohort,margen,adjust_retention,median_diff_days_adjusted,churned,churned_date,CLTV,CLTV_sin_margen,CLTV_sin_churn,cash_margin
0,VCP,54609840,Descartáveis,NaN,NaN,0.000000,NaN,0.000000,1,14.694730,...,NaN,0.218,0.941,NaN,1,2023-01-12,NaN,NaN,NaN,3.203451
11,VCP,135201529,Descartáveis,20.0,20.0,20.000000,20.0,0.000000,2,7.562100,...,NaN,0.218,0.941,18.250000,1,2022-07-18,NaN,NaN,NaN,1.648538
17,VCP,149306304,Descartáveis,5.0,111.0,60.600000,59.0,44.635188,6,28.615077,...,0.835865,0.218,1.000,6.186441,0,2023-08-01,9.677198,44.390816,11.577466,6.238087
24,VCP,149313616,Descartáveis,138.0,138.0,138.000000,138.0,0.000000,2,3.151570,...,NaN,0.218,0.997,2.644928,1,2023-03-05,NaN,NaN,NaN,0.687042
30,VCP,149317776,Descartáveis,14.0,14.0,14.000000,14.0,0.000000,2,12.614735,...,NaN,0.218,0.941,26.071429,1,2022-06-13,NaN,NaN,NaN,2.750012
36,VCP,149318063,Descartáveis,6.0,60.0,15.520000,13.0,11.839623,26,2.785825,...,1.000000,0.218,1.000,28.076923,0,2023-07-10,13.641114,62.573915,13.641114,0.607310
43,VCP,149327686,Descartáveis,4.0,4.0,4.000000,4.0,0.000000,2,5.076835,...,NaN,0.218,0.941,91.250000,1,2022-05-17,NaN,NaN,NaN,1.106750
49,VCP,149350490,Descartáveis,3.0,42.0,14.272727,11.0,13.023755,12,6.721221,...,0.051533,0.218,0.991,33.181818,1,2022-11-12,1.986335,9.111630,38.545038,1.465226
54,VCP,149373205,Descartáveis,5.0,191.0,45.000000,23.0,61.439866,9,17.832740,...,1.000000,0.218,1.000,15.869565,0,2023-10-22,12.338705,56.599566,12.338705,3.887537
61,VCP,149399006,Descartáveis,10.0,181.0,59.400000,28.0,69.923530,6,32.143152,...,1.000000,0.218,1.000,13.035714,0,2023-09-26,9.134395,41.900894,9.134395,7.007207


# Aca empieza el analisis global by customer 

In [277]:
#Here, i group the df3 (the df with the whole info by cat to customer lvl)
df_new3 = df3[['city','customer_id','aov_cat_usd','cash_margin','CLTV_sin_churn','churned']].groupby(by=['city','customer_id']).sum().reset_index()

In [278]:
#The processing data for customer level is made here

df_new = df.groupby(by=['city','customer_id','order_id','submit_date']).sum().reset_index()

df_new['days_bet'] = (df_new.submit_date -df_new.groupby('customer_id').submit_date.transform('min')).dt.days

#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df_new['res'] = df_new.groupby(by=['customer_id'])['days_bet'].diff()
df_grouped_2 = df_new.groupby(by=['city', 'customer_id']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).droplevel(0, axis="columns").reset_index()

In [279]:
#Finally we define everything in a df_new2 at level customer for further processing
df_new2 = df_grouped_2.copy()

df_new2.columns = ['city', 'customer_id', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [280]:
#Finally we calculate the next parameters and also integred with df_new3 where we can have the info grouped by customer about the category performance
df_new2["churned_ALL"] = [0 if ((x[12]) - (x[4]+x[6]*2) <= 0) else 1 for x in df_new2.values]

df_new2['adjust_std'] = pd.qcut(df_new2.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
df_new2[["adjust_std"]] = df_new2[["adjust_std"]].astype(float)

df_new2['adjust_retention'] = [1 if diff_month(x[10],x[13])+1 >= 10 else 
                                retention_aux[(retention_aux.region_code == x[0])][str(diff_month(x[10],x[13])+1)].max() for x in df_new2.values]

df_new2['median_diff_days_adjusted'] = 365/df_new2.median_diff_days

#this merge is to include the info for the total cash_margin, how many cats is churned and the total cat CLTV
df_new2 = df_new2.merge(df_new3[['customer_id','cash_margin','CLTV_sin_churn','churned']], on='customer_id', how='left')

In [281]:
#We calculate now the CLTV taking into account the global behavior of the customer
df_new2["ALL_CLTV_sin_churn"] = df_new2.median_diff_days_adjusted * df_new2.adjust_retention * df_new2.adjust_std * df_new2.cash_margin / 2
df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']] = df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']].fillna(0)

#df_new2["CLTV"] = df_new2.median_diff_days_adjusted * df_new2.margen * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort
#df_new2["CLTV_sin_margen"] = df_new2.median_diff_days_adjusted * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort

#If the Total Cat CLTV is NA because it is so few orders for each category, we create a Adjust_CLTV to include the global or ALL_CLTV_sin_churn instead of the NULL value
df_new2['Adjust_CLTV'] = [x[21] if x[19] == 0 else x[19] for x in df_new2.values]

print(df_new2.Adjust_CLTV.sum(),df_new2.CLTV_sin_churn.sum())

652743.0000702777 644055.1230137593


In [282]:
#Finally, we calculate the ALL_churned_date based on the general behavior of the customer
df_new2[['avg_diff_days','stdev_diff_days']] = df_new2[['avg_diff_days','stdev_diff_days']].fillna(0)
df_new2['ALL_churned_date'] = [(x[10] + datetime.timedelta(days=int(x[4]+x[6]*2))).strftime('%Y-%m-%d') for x in df_new2.values] #1 months

# Final processing of the file to export

In [283]:
#We will merge the df_new2 with the clear_buyers in order to filter by restaurants and take away kams (also to include the microsegment)
df_new2 = df_new2.merge(clear_buyers[clear_buyers.registered_city == city][['customer_id','business_type','microsegment_name','is_kam']], on='customer_id', how='left')
df_new2_end = df_new2[(df_new2.is_kam == 0)].copy() #df_new2[(df_new2.business_type == 'Restaurante') & (df_new2.is_kam == 0)].copy()

In [284]:
#This cell is to calculate the cumulative sum to create the tiers further
df_new2_end = df_new2_end.sort_values(['Adjust_CLTV'], ascending=[False])
df_new2_end['cumsum'] = df_new2_end.groupby('city')['Adjust_CLTV'].cumsum()
df_new2_end['run_pct'] = df_new2_end.groupby('city')['Adjust_CLTV'].apply(lambda x: (x/x.sum()).cumsum())

In [285]:
#This conditional create the tiers
df_new2_end['tier'] = ['A' if x <= 0.1 else
                       'B' if x <= 0.2 else
                       'C' if x <= 0.3 else
                       'D' if x <= 0.4 else
                       'E' if x <= 0.5 else
                       'F' if x <= 0.6 else
                       'G' if x <= 0.7 else
                       'H' if x <= 0.8 else
                       'I' if x <= 0.9 else
                       'J' for x in df_new2_end.run_pct.values]

In [286]:
df_new2_end.tier.value_counts()

J    2145
I     524
H     351
G     253
F     191
E     148
D     114
C      87
B      60
A      33
Name: tier, dtype: int64

In [287]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

# df3 = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders',
#        'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
#        'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
#        'median_diff_days_adjusted', 'churned', 'churned_date','CLTV_sin_churn']].copy()

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [288]:
df_new2_end.columns

Index(['city', 'customer_id', 'min_diff_days', 'max_diff_days',
       'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
       'ALL_num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date',
       'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'churned_ALL', 'adjust_std', 'adjust_retention',
       'median_diff_days_adjusted', 'cash_margin', 'TOTAL_CAT_CLTV_sin_churn',
       'churned', 'ALL_CLTV_sin_churn', 'Adjust_CLTV', 'ALL_churned_date',
       'business_type', 'microsegment_name', 'is_kam', 'cumsum', 'run_pct',
       'tier'],
      dtype='object')

In [289]:
#We merge with data_farming so we get the info farming for each customer
df_new2_end = df_new2_end[['customer_id','ALL_num_orders', 'aov_ALL_usd', 'churned_ALL', 'ALL_churned_date','TOTAL_CAT_CLTV_sin_churn',
       'ALL_CLTV_sin_churn', 'Adjust_CLTV','tier', 'microsegment_name', 'business_type', 'is_kam']].merge(data_farming[data_farming.ciudad == city][['customer_id','owner']], on='customer_id', how='left')
df_new2_end[['owner']] = df_new2_end[['owner']].fillna('No Farming')

In [290]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

# df3 = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders',
#        'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
#        'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
#        'median_diff_days_adjusted', 'churned', 'churned_date','CLTV_sin_churn']].copy()

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [291]:
df_final = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders_cat',
       'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
       'median_diff_days_adjusted', 'churned_cat', 'churned_date','CAT_CLTV_sin_churn']].merge(df_new2_end, on='customer_id', how='inner')

In [292]:
df_final.head(10)

,city,customer_id,cat,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders_cat,aov_cat_usd,...,churned_ALL,ALL_churned_date,TOTAL_CAT_CLTV_sin_churn,ALL_CLTV_sin_churn,Adjust_CLTV,tier,microsegment_name,business_type,is_kam,owner
0,VCP,54609840,Descartáveis,NaN,NaN,0.000000,NaN,0.000000,1,14.694730,...,1,2023-01-30,143.719302,174.899213,143.719302,H,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
1,VCP,54609840,Frutas e Verduras,1.0,28.0,12.833333,14.0,7.286167,19,18.958205,...,1,2023-01-30,143.719302,174.899213,143.719302,H,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
2,VCP,54609840,Laticínios e Ovos,1.0,35.0,14.437500,10.5,11.033094,17,28.226991,...,1,2023-01-30,143.719302,174.899213,143.719302,H,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
3,VCP,54609840,Limpeza e Higiene,11.0,71.0,37.000000,29.0,30.789609,4,6.606835,...,1,2023-01-30,143.719302,174.899213,143.719302,H,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
4,VCP,54609840,Mercearia,1.0,28.0,7.483871,7.0,6.249645,32,44.171306,...,1,2023-01-30,143.719302,174.899213,143.719302,H,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
5,VCP,135201529,Descartáveis,20.0,20.0,20.000000,20.0,0.000000,2,7.562100,...,0,2023-07-06,460.153569,521.300043,460.153569,D,Pratos/Refeições,Restaurante,False,No Farming
6,VCP,135201529,Frutas e Verduras,1.0,22.0,3.820755,3.0,3.460793,107,25.655630,...,0,2023-07-06,460.153569,521.300043,460.153569,D,Pratos/Refeições,Restaurante,False,No Farming
7,VCP,135201529,Laticínios e Ovos,1.0,153.0,26.600000,8.0,39.908467,16,19.593204,...,0,2023-07-06,460.153569,521.300043,460.153569,D,Pratos/Refeições,Restaurante,False,No Farming
8,VCP,135201529,Limpeza e Higiene,2.0,84.0,15.363636,10.5,17.431623,23,6.542877,...,0,2023-07-06,460.153569,521.300043,460.153569,D,Pratos/Refeições,Restaurante,False,No Farming
9,VCP,135201529,Mercearia,1.0,44.0,5.867647,2.5,8.437486,69,29.332868,...,0,2023-07-06,460.153569,521.300043,460.153569,D,Pratos/Refeições,Restaurante,False,No Farming


In [293]:
df_final.to_excel(f"CLTV_{city}.xlsx",index=False)

# Info for SAC team

In [294]:
df_SAC = df_final[['city','customer_id','tier','Adjust_CLTV']].drop_duplicates()

In [295]:
letter_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10}
df_SAC['tier'] = df_SAC['tier'].map(letter_to_number)

In [296]:
df_SAC.to_csv(f"{city}_SAC_tiers.csv",index=False)